In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import re
import datetime
import folium
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas import GeoDataFrame
import math
from folium.plugins import HeatMap, MarkerCluster
from folium import Marker,GeoJson,Choropleth, Circle

### Import datasets

In [ ]:
time_prov = pd.read_csv('/kaggle/input/coronavirusdataset/TimeProvince.csv')
weather = pd.read_csv('/kaggle/input/coronavirusdataset/Weather.csv')

# 1. The first analysis is a time series of the Covid-19 cases in the Province of South Korea

In [ ]:
time_prov.head()

## Because the dates in the data are repeating, I'll group the data by date to see the chrological timeline of the other columns

In [ ]:
time = time_prov.groupby('date', as_index = False).sum()

In [ ]:
time

## A plot of the released and confirmed columns show in the beginning of the pandemic there are more confirmed cases than are released patients, and then the released patients became more than there were confirmed cases. The curiously the cycle starts over again.

In [ ]:
sns.scatterplot('released', 'confirmed', data = time)

## Highlighting the deceased in the above graph, it shows as time went on the number of deaths increased.

In [ ]:
sns.scatterplot('released', 'confirmed', data = time, hue = 'deceased')


## A plot of the deceased and the confirmed show that in the beginning the number of confirmed cases were increasing more than deaths, but then it got to a point where the number deaths was greater. Then it looks like measures where taken to curb the deaths, so the number of deaths slowed down while to confirmed cases increased

In [ ]:
sns.scatterplot('deceased', 'confirmed', data = time)
 

## This shows that the number of deaths in the beginning was a lot. Probably hospitals didn't know how to handle things for a while and then some kind of intervention helped. That increased the number of patients who were able to be released. However, the number of deaths stayed in tandem with the released patients until it got to a point where the deaths was stayed and the released patients increased

In [ ]:
sns.scatterplot('deceased', 'released', data = time)

# Now looking at the data with respect to the provinces. It shows certain provinces have corresponding number of deaths, confirmed cases and released patients. Except for Seoul. Seoul has kept their deceased in check

In [ ]:
time_prov.head()

In [ ]:
# Daegu and certain provinces have high confirmed values

sns.barplot('confirmed','province', data = time_prov) 

In [ ]:
# They also have high released values

sns.barplot('released','province',  data = time_prov) 

In [ ]:
# Gyeongsangbuk-do is the only province with disproportionally high deceased number of deaths. 
# Seoul, though, has lower deaths than expected. The question for both is why?

sns.barplot('deceased','province', data = time_prov)

## The time of the testing was done only at 12am and 4pm

In [ ]:
sns.scatterplot('time', 'confirmed', data = time_prov)

# 2. Moving on to the Weather analysis of regions in South Korea. 
## Again the dates here are repeated, so I'll group them to see the timeline of what's happening with the weather

In [ ]:
weather.head()

In [ ]:
# Grouping...

date = weather.groupby('date', as_index = False).sum() 

In [ ]:
date

In [ ]:
# The average temperature increases the average relative humidity  

sns.scatterplot('avg_temp', 'avg_relative_humidity', data = date)

In [ ]:
# The most wind direction decreases the average temperature
sns.scatterplot('avg_temp', 'most_wind_direction', data = date)

In [ ]:
# The maximum wind speed decreases the average temperature slightly
sns.scatterplot('avg_temp', 'max_wind_speed', data = date)

In [ ]:
# This makes sense that it's a one-to-one correlation

sns.scatterplot('avg_temp', 'min_temp', data = date) 

In [ ]:
# Same thing here

sns.scatterplot('avg_temp', 'max_temp', data = date) 

In [ ]:
# Precipitation has no bearing on the average temperature

sns.scatterplot('avg_temp', 'precipitation', data = date)

## Grouping the data by province will reveal the weather patterns there

In [ ]:

province = weather.groupby(['province'], as_index = False).sum()

In [ ]:
province

In [ ]:
sns.barplot('avg_temp', 'province', data = province)

In [ ]:
sns.barplot('max_wind_speed', 'province', data = province)

In [ ]:
sns.barplot('precipitation', 'province', data = province)

In [ ]:
sns.barplot('most_wind_direction', 'province', data = province)

In [ ]:
sns.barplot('avg_relative_humidity', 'province', data = province)

## It looks like the weather pattern is not corresponding to any of the provinces, so weather doesn't seem to be playing a role in transmission. 